<a href="https://colab.research.google.com/github/Diwakar-Gupta/Data-Science-APRIL/blob/main/22-07-16-ObjectDetection/car-object-localization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

import numpy as np 
import pandas as pd 
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import math
import cv2
from PIL import Image
  

print('Tensorflow Version:', tf.__version__)
print('Keras Version:', keras.__version__)

In [ ]:
tf.config.list_physical_devices()

In [ ]:
image_root = '../input/car-object-detection/data/training_images'
BATCH_SIZE = 16
IMAGE_SIZE = (380, 676)

In [ ]:
df = pd.read_csv('../input/car-object-detection/data/train_solution_bounding_boxes (1).csv')
df.sample(4)

In [ ]:
print('Path to an image ', os.path.join(image_root,'vid_4_6440.jpg'))

In [ ]:
def draw_box(img, xy):
    x1, y1, x2, y2 = xy.astype('int16')
    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), 5)
    return img

In [ ]:
plt.figure(figsize=(20, 12))

for i, (index, row) in enumerate(df.head(4).iterrows()):
    plt.subplot(2, 2, i+1)
    
    img = Image.open(os.path.join(image_root,row['image']))
    img = np.asarray(img)
    img = img/255
    print(img.shape)
    
    img = draw_box(img, np.array([row['xmin'], row['ymin'], row['xmax'], row['ymax']]))
    plt.imshow(img)

# Creating DataSet

In [ ]:
# step 1
filenames = tf.constant(df['image'].map(lambda x: os.path.join(image_root,x)))
labels = tf.constant(df[['xmin', 'ymin', 'xmax', 'ymax']])

In [ ]:
#step 2
dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))

In [ ]:
def load_img(filename, label):
    image_string = tf.io.read_file(filename)
    image_decoded = tf.image.decode_jpeg(image_string, channels=3)
    image = tf.cast(image_decoded, tf.float32)

    # Do preprocessing here
    image = keras.applications.xception.preprocess_input(image)

    return image, label

dataset = dataset.map(load_img).batch(BATCH_SIZE)

In [ ]:
plt.figure(figsize=(20, 12))


for x_batch, y_batch in dataset.take(1):
    height = int(math.sqrt(BATCH_SIZE))
    for i, (image, label) in enumerate(zip(x_batch, y_batch)):
        plt.subplot(height, height, i+1)
        
        img = image.numpy() # [-1, 1]
        img = img+1 # [0, 2]
        img = img/2 # [0, 1]
        
        img = draw_box(img, np.array([*label]))
        
        plt.imshow(img)
        plt.title(img.shape)

plt.show()

# Make and Train Model

In [ ]:
# for both classification and localization

model = ???

model.compile(loss=["sparse_categorical_crossentropy", "mse"],loss_weights=[0.5, 0.5],
              optimizer='adam', metrics=[tf.keras.metrics.MeanIoU(num_classes=n_classes)])

In [ ]:
# for both only localization

model = ???

model.compile(loss=["mse"],
              optimizer='adam', metrics=[tf.keras.metrics.MeanIoU(num_classes=n_classes))

In [ ]:
val_dataset = dataset.take(4)
train_dataset = dataset.skip(4)

# See Evaluation Result

In [ ]:
plt.figure(figsize=(15, 8))

for x_batch, y_batch in val_dataset.take(3):
    for i in range(6):
        plt.subplot(2, 3, i+1)
        
        image = x_batch[i]
        img = image.numpy()
        label = y_batch[i]
        
        img = img+1
        img = img/2
        xy = model(image[tf.newaxis, :]).numpy()[0]
        plt.title("{}".format(xy))
        
        img = draw_box(img, xy)
        
        plt.imshow(img)
        

plt.show()